In [29]:
import pandas as pd
train_fear_df = pd.read_csv("feartrainalldata6.txt","\t")
test_fear_df = pd.read_csv("feartestalldata6.txt","\t")

In [32]:
from konlpy.tag import Tw
Twitter_fear = Twitter()
Twitter_fear.pos(u'오늘 하루도 수고했어')

C:\Users\sunho\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


[('오늘', 'Noun'), ('하루', 'Noun'), ('도', 'Josa'), ('수고', 'Noun'), ('했어', 'Verb')]

In [34]:
from konlpy.tag import Hannanum
Twitter_fear = Hannanum()
Twitter_fear.pos(u'오늘 하루도 수고했어')

[('오늘', 'N'), ('하루', 'N'), ('도', 'J'), ('수고', 'N'), ('하', 'X'), ('었어', 'E')]

In [4]:
def tokenize(doc):
    return ['/'.join(t) for t in okt_fear.pos(doc, norm=True, stem=True)]

In [5]:
train_fear_df.isnull().any()
train_fear_df['document'] = train_fear_df['document'].fillna('');
train_fear_df.isnull().any()
test_fear_df['document'] = test_fear_df['document'].fillna('');

In [6]:
train_fear_docs = [(tokenize(row[1]), row[2]) for row in train_fear_df.values]
test_fear_docs = [(tokenize(row[1]),row[2]) for row in test_fear_df.values]

In [7]:
tokens = [t for d in train_fear_docs for t in d[0]]
print("토큰개수:",len(tokens))

토큰개수: 437889


In [8]:
import nltk
fear_text = nltk.Text(tokens, name='NMSC')
print(len(fear_text.tokens))
print(len(set(fear_text.tokens)))
print(fear_text.vocab().most_common(10))

437889
20799
[('하다/Verb', 15082), ('이/Josa', 8604), ('./Punctuation', 7099), ('?/Punctuation', 6761), ('가/Josa', 6624), ('../Punctuation', 5995), ('에/Josa', 5408), ('.../Punctuation', 4332), ('들/Suffix', 4102), ('도/Josa', 3987)]


In [10]:
FEAR_FREQUENCY_COUNT = 3000;
fear_selected_words = [f[0] for f in fear_text.vocab().most_common(FEAR_FREQUENCY_COUNT)]

In [11]:
def fear_term_frequency(doc):
    return [doc.count(word) for word in fear_selected_words]

In [12]:
x_fear_train = [fear_term_frequency(d) for d,_ in train_fear_docs]
x_fear_test = [fear_term_frequency(d) for d,_ in test_fear_docs]
y_fear_train = [c for _,c in train_fear_docs]
y_fear_test = [c for _,c in test_fear_docs]

In [13]:
import numpy as np
x_fear_train = np.asarray(x_fear_train).astype('float32')
x_fear_test = np.asarray(x_fear_test). astype('float32')

y_fear_train = np.asarray(y_fear_train).astype('float32')
y_fear_test = np.asarray(y_fear_test).astype('float32')

In [14]:
import tensorflow as tf
fear_model= tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation = 'relu',input_shape=(FEAR_FREQUENCY_COUNT,)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [15]:
fear_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
             loss=tf.keras.losses.binary_crossentropy,
             metrics=[tf.keras.metrics.binary_accuracy]
             )

In [16]:
fear_model.fit(x_fear_train, y_fear_train, epochs=50, batch_size=512)

Epoch 1/50
91/91 [==============================] - ETA: 0s - loss: 0.5075 - binary_accuracy: 0.766 - 1s 8ms/step - loss: 0.5016 - binary_accuracy: 0.7696
Epoch 2/50
91/91 [==============================] - 1s 8ms/step - loss: 0.3343 - binary_accuracy: 0.8601
Epoch 3/50
91/91 [==============================] - 1s 8ms/step - loss: 0.2543 - binary_accuracy: 0.9012
Epoch 4/50
91/91 [==============================] - 1s 7ms/step - loss: 0.1855 - binary_accuracy: 0.9329
Epoch 5/50
91/91 [==============================] - 1s 7ms/step - loss: 0.1275 - binary_accuracy: 0.9573
Epoch 6/50
91/91 [==============================] - 1s 7ms/step - loss: 0.0845 - binary_accuracy: 0.9736
Epoch 7/50
91/91 [==============================] - 1s 7ms/step - loss: 0.0549 - binary_accuracy: 0.9841
Epoch 8/50
91/91 [==============================] - 1s 8ms/step - loss: 0.0362 - binary_accuracy: 0.9898
Epoch 9/50
91/91 [==============================] - 1s 8ms/step - loss: 0.0236 - binary_accuracy: 0.9933
Epoch

In [17]:
fear_results = fear_model.evaluate(x_fear_test, y_fear_test)

619/619 [==============================] - 1s 1ms/step - loss: 10.2607 - binary_accuracy: 0.6062


In [18]:
fear_results

[10.2607421875, 0.6062061190605164]

In [19]:
fear_model.save('fear_model.h5')

In [20]:
fear_review = "너무 무섭다"
fear_token = tokenize(fear_review)
fear_token

['너무/Adverb', '무섭다/Adjective']

In [21]:
tf = fear_term_frequency(fear_token)
data = np.expand_dims(np.asarray(tf).astype('float32'),axis=0)
float(fear_model.predict(data))

0.9960774779319763

In [22]:
def fear_predict(predict):
    fear_token = tokenize(predict)
    fear_tfq = fear_term_frequency(fear_token)
    fear_data = np.expand_dims(np.asarray(fear_tfq).astype('float32'), axis=0)
    fear_score = float(fear_model.predict(fear_data))
    if(fear_score > 0.5):
        print(f"{predict} ==> 공포 ({round(fear_score*100)}%)")
    else:
        print(f"{predict} ==> 공포X ({round((1-fear_score)*100)}%)")

In [26]:
fear_predict("")

안 무서워ㅋㅋ ==> 공포 (100%)
